In [1]:
import sqlite3
from datetime import date, timedelta, datetime
import pandas as pd
import requests
from google.transit import gtfs_realtime_pb2
import ipywidgets as widgets
import toml

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
today = date.today().strftime('%Y%m%d')
tomorrow = (date.today() + timedelta(days=1)).strftime('%Y%m%d')

def make_timestamp(date_str, time_str):
    timestamp = pd.to_datetime(date_str)
    hours = int(time_str[:2])
    if hours > 23:
        timestamp += timedelta(days=1)
        hours = hours - 24

    timestamp += timedelta(hours=hours,minutes=int(time_str[3:5]), seconds=int(time_str[6:8]))
    return timestamp

def make_in_params(count):
    return ','.join([ '?' ] * count)


In [2]:
config_file = widgets.Dropdown(
    options=['delijn.toml', 'sncb.toml', 'sncb_zuid.toml'],
    description='Configuration:',
)

display(config_file)

Dropdown(description='Configuration:', options=('delijn.toml', 'sncb.toml', 'sncb_zuid.toml'), value='delijn.t…

In [3]:
with open(config_file.value) as f:
    config = toml.load(f)


In [9]:
with sqlite3.connect(config['gtfs']['database']) as con:

    trip_query = (
        'SELECT st1.trip_id, st1.stop_id, s.stop_name, r.route_short_name AS route, t.trip_headsign AS destination, '
        'r.route_color as color, c.date, st1.departure_time '
        'FROM stop_times st1 INNER JOIN stop_times st2 ON st1.trip_id = st2.trip_id '
        'INNER JOIN stops s ON s.stop_id = st1.stop_id '
        'INNER JOIN trips t ON t.trip_id = st1.trip_id '
        'INNER JOIN routes r ON t.route_id = r.route_id '
        'INNER JOIN calendar_dates c ON c.service_id = t.service_id '
        f'WHERE st1.stop_id IN ({make_in_params(len(config["gtfs"]["departure_stops"]))}) AND '
        f'st2.stop_id IN ({make_in_params(len(config["gtfs"]["arrival_stops"]))}) '
        'AND CAST(st2.stop_sequence AS INTEGER) > CAST(st1.stop_sequence AS INTEGER) '
        'AND c.exception_type = 1 '
        f'AND c.date IN ({today}, {tomorrow}) '
        'ORDER BY c.date, st1.departure_time'
    )

trips = pd.read_sql_query(trip_query, con, params=config["gtfs"]['departure_stops'] + config["gtfs"]['arrival_stops'])
trips['timestamp'] = trips.apply(lambda row: make_timestamp(row.date, row.departure_time), axis=1)
trips['cancelled'] = False
trips['skipped'] = False
trips['delay'] = 0

trips = trips[trips['timestamp'] > datetime.now()]

#trips
trips.shape[0]

145

In [10]:
hdr ={
    'Cache-Control': 'no-cache',
}

if config['gtfs']['realtime_key'] != '':
    hdr['Ocp-Apim-Subscription-Key'] = config['gtfs']['realtime_key']


In [11]:
response = requests.get(config['gtfs']['realtime_url'], headers=hdr)
feed = gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(response.content)

for entity in feed.entity:
    trip_id = entity.trip_update.trip.trip_id
    if trip_id in trips['trip_id']:
        print(f'GOT A TRIP! {trip_id}')
        if entity.HasField('trip_update'):
            if entity.trip_update.trip.schedule_relationship == 3:
                trips.loc[trips['trip_id'] == trip_id, 'cancelled'] = True
            else:
                stop_id = trips[trips['trip_id'] == trip_id]['stop_id'].iloc[0]
                for stop_time_update in entity.trip_update.stop_time_update:
                    if stop_time_update.stop_id in departure_stops:
                        if stop_time_update.schedule_relationship == 1:
                            trips.loc[trips['trip_id'] == trip_id, 'skipped'] = True
                        else:
                            trips.loc[trips['trip_id'] == trip_id, 'delay'] = int(stop_time_update.departure.delay) if stop_time_update.HasField("departure") else 0

#trips